In [39]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional,Dict,Tuple, Literal
from pathlib import Path
from enefit_challenge.utils.dataset import load_enefit_training_data
from enefit_challenge.models.forecaster import Forecaster

import warnings
warnings.filterwarnings('ignore')

In [40]:
class XGBoostForecaster(Forecaster):
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri("http://127.0.0.1:5000/")
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> XGBRegressor:
        """
        Trains a `XGBRegressor` and tracks it using mlflow
        """
        model = XGBRegressor(
            n_estimators=100, 
            objective='reg:squarederror'
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        metrics: list=["mae"]
    ) -> float:
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        mae = mean_absolute_error(y_test, y_test_pred)

        mlflow.xgboost.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        params: Optional[Dict]=None,
        metrics: list=["MAE"]
    ) -> Dict:
        """ 
        Takes an instance of `XGBRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `metrics`: `list`
            list of the metrics to track in the mlflow experiment run.
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name
        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))

        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            # tracking_uri=self.tracking_uri,
            metric_name="MAE",
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 50, 200, log=True),
                'eta': trial.suggest_float('eta', 0.01, 0.95,log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 10, log=True),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 25, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1, log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 1, log=True),
                'colsample_bynode': trial.suggest_float("colsample_bynode", 0.1, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 1, log=True),
                'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
                'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True)
                # 'l2_leaf_reg': trial.suggest_float('l2_leaf_reg',1e-8,100,log=True),
                # 'model_size_reg': trial.suggest_float('model_size_reg',1e-8,100,log=True),
            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index]
                )
            trial.set_user_attr('split_mae', cv_mae)
            return np.mean(cv_mae)

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=42
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=100, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches the latest version of the model from the mlflow backend and uses it
        to perform prediction on new input data.
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                "You must specify which kind of XGBoostForecaster you intend to use for prediction"
            )


In [41]:
df_train = load_enefit_training_data()

In [42]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id',
                       'county', 'product_type']

xgbf = XGBoostForecaster()

xgbf.train_model(
    train_df=df_train,
    target_col="target",
    model_name="xgboost_enefit",
    exclude_cols=not_feature_columns,
)

[I 2023-11-18 13:33:49,054] A new study created in memory with name: xgboost
[I 2023-11-18 13:34:11,100] Trial 0 finished with value: 203.6244684794037 and parameters: {'n_estimators': 84, 'eta': 0.7590145927293601, 'max_depth': 5, 'min_child_weight': 5, 'colsample_bytree': 0.1432249371823025, 'colsample_bylevel': 0.14321698289111517, 'colsample_bynode': 0.1143098387631322, 'subsample': 0.9114125527116832, 'lambda': 0.25378155082656645, 'alpha': 0.6796578090758157}. Best is trial 0 with value: 203.6244684794037.
[I 2023-11-18 13:34:28,368] Trial 1 finished with value: 205.82206414006154 and parameters: {'n_estimators': 51, 'eta': 0.8283494908181351, 'max_depth': 6, 'min_child_weight': 1, 'colsample_bytree': 0.1519934830130981, 'colsample_bylevel': 0.15254729458052607, 'colsample_bynode': 0.20148477884158655, 'subsample': 0.7193453335958095, 'lambda': 0.05342937261279776, 'alpha': 0.014618962793704969}. Best is trial 0 with value: 203.6244684794037.
[I 2023-11-18 13:34:44,396] Trial 2 f

KeyboardInterrupt: 

In [20]:
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id', 'county', 'product_type']

# predict on training data is done only to test the forecast method
X = df_train.drop(['target'] + not_feature_columns, axis=1).tail(5)
y = df_train['target'].tail(5)

In [21]:
xgbf.forecast(
    input_data=X,
    use_best_from_run=False,
    use_env_model="Staging",
)

array([756.55133 ,  27.508097,  49.8362  ,   5.408032, 332.51645 ],
      dtype=float32)

In [22]:
y

2017819    702.729
2017820     45.864
2017821     25.221
2017822     12.243
2017823    196.240
Name: target, dtype: float64

In [34]:
xgbf = XGBoostForecaster()
xgbf.forecast(
    input_data=X,
    use_best_from_run=False,
    use_env_model=None,
    use_version=None
)

ValueError('You must specify which kind of XGBoostForecaster yoiu intend to use for prediction')